In [1]:
# utilities
import numpy as np
import pandas as pd

# plotting
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.neural_network import MLPClassifier

import gensim

In [2]:
df = pd.read_csv('twitter.cleaned_no_hashtags.csv')
df


,text,target
0,love not wait se admin best server ever hapy,1.0
1,recruiter comunity server sharepoint dev gig a...,1.0
2,not paying god seat watch miley gues bc switch...,0.0
3,ah excited pushi home okay aw hapy right,1.0
4,col pic amp username ahah whats up,1.0
...,...,...
1545656,going go big house borow si guitar play maybe ...,1.0
1545657,dish sen prepare website would afraid recomend...,1.0
1545658,birthday,1.0
1545659,hury not think jon kate,1.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1545661 entries, 0 to 1545660
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   text    1545661 non-null  object 
 1   target  1545661 non-null  float64
dtypes: float64(1), object(1)
memory usage: 23.6+ MB


### Analyzing sequences to use Word2Vec

In [4]:
val = max(df['text'].apply(lambda x: len(x.split())))
val

34

In [5]:
max_sequence_length = (int(val/4)+1)*4
print(f'Padding length is {max_sequence_length}')

Padding length is 36


#### Max sequence length is {{val}}, assume {{max_sequence_length}} to keep sufficient padding

In [6]:
# data = [alp.split() for alp in df['text']]

# # corpus_text=[elem for elem in alp for alp in data]

In [7]:
model_name = r'/Users/subhagr/Desktop/My files/GoogleNews-vectors-negative300.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(model_name, binary=True)

In [8]:
word = 'king'

In [9]:
word_list = model.index_to_key

print(len(word_list))

3000000


In [10]:
len(model.key_to_index.keys())

3000000

In [11]:
model.most_similar(word)

[('kings', 0.7138046622276306),
 ('queen', 0.6510956287384033),
 ('monarch', 0.6413194537162781),
 ('crown_prince', 0.6204220056533813),
 ('prince', 0.6159993410110474),
 ('sultan', 0.5864824056625366),
 ('ruler', 0.5797566771507263),
 ('princes', 0.5646552443504333),
 ('Prince_Paras', 0.5432944297790527),
 ('throne', 0.5422104597091675)]

In [12]:
word_list[:10]

['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said']

In [13]:
def word_checker(word):
    for letter in word:
        if letter not in 'abcdefghijklmnopqrstuvwxyz':
            return False
        return True

In [14]:
word_list = [word for word in word_list if word_checker(word)]

In [15]:
len(word_list)

631815

In [16]:
model['list'].shape

(300,)

In [17]:
vector_size = model['word'].shape[0]

## Splitting, vectorising, and flattening the data

In [18]:
minidf = df.sample(frac=0.01)
minidf['target'].value_counts()

1.0    7744
0.0    7713
Name: target, dtype: int64

Balanced!

In [19]:
X_train, X_test, y_train, y_test = train_test_split(minidf['text'].apply(lambda x: x.split()), 
                                    minidf['target'].values, test_size=0.2, random_state=42)

In [20]:
# this function is passed with a sequence- list of words and returns the vector
# If normal = True, vector will be averaged over all words, otherwise summed

def get_embedding(sequence, normal = False):
    vec = np.zeros((vector_size), dtype= float)
    count= 0
    for word in sequence:
        if word in word_list:
            vec += model[word]
            count +=1
#             print(word)
    global glob_counter
    glob_counter +=1
    if(glob_counter%100 ==0):print(glob_counter)
        
    if(normal): return vec/(max(count,1))
    else: return vec

In [21]:
glob_counter = 0
X_train_embed = np.array([get_embedding(sequ) for sequ in X_train])
X_test_embed = np.array([get_embedding(sequ) for sequ in X_test])

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400


In [22]:
X_train_embed.shape

(12365, 300)

In [23]:
X_test_embed.shape

(3092, 300)

## Using w2v embeddings on models to check performance

In [24]:
from sklearn.svm import LinearSVC

svc = LinearSVC(C = 0.1)
svc.fit(X_train_embed, y_train)


/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(C=0.1)

In [25]:
y_pred = svc.predict(X_test_embed)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.72      0.71      0.71      1515
         1.0       0.72      0.73      0.73      1577

    accuracy                           0.72      3092
   macro avg       0.72      0.72      0.72      3092
weighted avg       0.72      0.72      0.72      3092

[[1070  445]
 [ 420 1157]]


In [26]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1)
lr.fit(X_train_embed, y_train)

LogisticRegression(C=1)

In [27]:
y_pred2 = lr.predict(X_test_embed)
print(classification_report(y_test, y_pred2))
print(confusion_matrix(y_test, y_pred2))

              precision    recall  f1-score   support

         0.0       0.72      0.71      0.72      1515
         1.0       0.73      0.73      0.73      1577

    accuracy                           0.72      3092
   macro avg       0.72      0.72      0.72      3092
weighted avg       0.72      0.72      0.72      3092

[[1078  437]
 [ 420 1157]]


In [28]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
bnb.fit(X_train_embed,y_train)


BernoulliNB()

In [29]:
y_pred3 = bnb.predict(X_test_embed)
print(classification_report(y_test, y_pred3))
print(confusion_matrix(y_test, y_pred3))

              precision    recall  f1-score   support

         0.0       0.65      0.67      0.66      1515
         1.0       0.67      0.65      0.66      1577

    accuracy                           0.66      3092
   macro avg       0.66      0.66      0.66      3092
weighted avg       0.66      0.66      0.66      3092

[[1010  505]
 [ 552 1025]]


## Using "better" methods for Word2Vec

In [30]:
mlp = MLPClassifier(hidden_layer_sizes=(300,10,10,1),activation='identity',solver='adam',
    alpha=0.0001, batch_size='auto',learning_rate='constant',learning_rate_init=0.001,power_t=0.5,
    max_iter=100,shuffle=True,random_state=None,tol=0.0001,verbose=True,warm_start=False,momentum=0.9,
    nesterovs_momentum=True, early_stopping=True,validation_fraction=0.1,beta_1=0.9,beta_2=0.999,
    epsilon=1e-08,n_iter_no_change=10,max_fun=15000)
mlp.fit(X_train_embed, y_train)

Iteration 1, loss = 0.65658856
Validation score: 0.688763
Iteration 2, loss = 0.57020640
Validation score: 0.711399
Iteration 3, loss = 0.55270267
Validation score: 0.733226
Iteration 4, loss = 0.55066642
Validation score: 0.722716
Iteration 5, loss = 0.54888045
Validation score: 0.730800
Iteration 6, loss = 0.54888306
Validation score: 0.725141
Iteration 7, loss = 0.54785144
Validation score: 0.715441
Iteration 8, loss = 0.54547799
Validation score: 0.729184
Iteration 9, loss = 0.54403480
Validation score: 0.730800
Iteration 10, loss = 0.54719617
Validation score: 0.718674
Iteration 11, loss = 0.54308648
Validation score: 0.717057
Iteration 12, loss = 0.54234802
Validation score: 0.717057
Iteration 13, loss = 0.54058466
Validation score: 0.728375
Iteration 14, loss = 0.54326757
Validation score: 0.724333
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPClassifier(activation='identity', early_stopping=True,
              hidden_layer_sizes=(300, 10, 10, 1), max_iter=100, verbose=True)

In [31]:
y_pred = mlp.predict(X_test_embed)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.72      0.71      0.72      1515
         1.0       0.72      0.74      0.73      1577

    accuracy                           0.72      3092
   macro avg       0.72      0.72      0.72      3092
weighted avg       0.72      0.72      0.72      3092

[[1073  442]
 [ 413 1164]]
